In [1]:
import datetime
import pandas as pd
import requests
import io
import json

url = 'http://gis.sutran.gob.pe/alerta_sutran/script_cgm/carga_xlsx.php'
headers = {'Accept': '/',
'Accept-Encoding': 'gzip, deflate',
'Accept-Language': 'en,es;q=0.9,en-GB;q=0.8',
'Connection': 'keep-alive',
'Content-Length': '9',
'Content-type': 'application/x-www-form-urlencoded; charset=UTF-8-SIG',
'Host': 'gis.sutran.gob.pe',
'Origin': 'http://gis.sutran.gob.pe',
'Referer': 'http://gis.sutran.gob.pe/alerta_sutran/',
'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36'}
body = {'tipo':'MAPA'}
req = requests.post(url, headers=headers, data=body)
now = datetime.datetime.now()

my_json = json.load(io.BytesIO(req.content))

restringido = {}
restringido['data'] = my_json['restringido']

interrumpido = {}
interrumpido['data'] = my_json['interrumpido']

with open('restringido.json', 'w') as f:
    json.dump(restringido, f)

with open('interrumpido.json', 'w') as f:
    json.dump(interrumpido, f)

df_r = pd.read_json('restringido.json', encoding = 'utf-8-sig', orient='split')
df_restringido = pd.json_normalize(df_r['properties'])

df_i = pd.read_json('interrumpido.json', encoding = 'utf-8-sig', orient='split')
df_interrumpido = pd.json_normalize(df_i['properties'])

frames = [df_restringido, df_interrumpido]
df_total = pd.concat(frames, ignore_index=True)
df_total = df_total[df_total['motivo'] == 'HUMANO']
df_total = df_total.replace('TRANSITO', 'TRÁNSITO', regex=True)
df_total = df_total.replace('"', '', regex=True)
df_total['style'] = df_total['estado'].str.split(' ').str[1].str.lower()
df_total.to_csv(f'{now}.csv', index=False)
df_total

,item,visualizacion_,tipo_alerta_,estado,fecha_evento,fecha_actualizacion,ubigeo,afectacion,evento,latitud,...,_sort_,_fecha_actualizacion_,cant_vehiculos_detenidos_pasajeros,cant_vehiculos_detenidos_mercancias,fuente,motivo,codigo_via,nombre_carretera,alerta_activa_24_horas,style
2,144,SI,VIGENTE,TRÁNSITO RESTRINGIDO,11/01/2023,23/01/2023 18:00 HORAS,CUSCO/LA CONVENCION/SANTA ANA,KM 199,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.873069617394,...,-12.873069617394+-72.692213162262,2023 18:00 HORAS+01+23+18:00,0,0,COE-MTC,HUMANO,PE-28B,CARRETERA QUINUA - QUILLABAMBA - CUSCO,SI,restringido
3,145,SI,VIGENTE,TRÁNSITO RESTRINGIDO,07/01/2023,23/01/2023 18:00 HORAS,CUSCO/LA CONVENCION/SANTA ANA,KM 230,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-12.880884243624,...,-12.880884243624+-72.690784623809,2023 18:00 HORAS+01+23+18:00,0,1,COE-MTC,HUMANO,PE-28B,CARRETERA QUINUA - QUILLABAMBA - CUSCO,SI,restringido
4,158,SI,VIGENTE,TRÁNSITO RESTRINGIDO,19/01/2023,23/01/2023 18:00 HORAS,CUSCO/URUBAMBA/MARAS,KM 44,PUENTE URUBAMBA. PARO INDEFINIDO SOLICITANDO N...,-13.312531099811,...,-13.312531099811+-72.117755850561,2023 18:00 HORAS+01+23+18:00,0,0,COE-MTC,HUMANO,PE-28F,CARRETERA POROY - CHINCHERO - URUBAMBA,SI,restringido
5,77,SI,VIGENTE,TRÁNSITO RESTRINGIDO,23/01/2023,23/01/2023 18:00 HORAS,AREQUIPA/CAMANA/SAMUEL PASTOR,KM 838,PARO INDEFINIDO SOLICITANDO NUEVAS ELECCIONES ...,-16.5972789,...,-16.5972789+-72.70454071,2023 18:00 HORAS+01+23+18:00,0,0,COE-MTC,HUMANO,PE-1S,LONGITUDINAL DE LA COSTA SUR,SI,restringido
8,169,SI,VIGENTE,TRÁNSITO RESTRINGIDO,22/01/2023,23/01/2023 18:00 HORAS,HUANUCO/HUANUCO/CHURUBAMBA,KM 25,PUENTE RANCHO PARO INDEFINIDO SOLICITANDO NUEV...,-9.831249,...,-9.831249+-76.096243,2023 18:00 HORAS+01+23+18:00,0,0,CEOPOL,HUMANO,PE-18A,CARRETERA HUANUCO - TINGO MARIA,SI,restringido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,333,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,21/01/2023,23/01/2023 18:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 390,SECTOR HUIPOCA. PARO INDEFINIDO SOLICITANDO NU...,-8.976077,...,-8.976077+-75.44483,2023 18:00 HORAS+01+23+18:00,0,0,UD UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
91,335,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,23/01/2023 18:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 406,SECTOR BARRIO UNIDO -AGUAYTIA PARO INDEFINIDO ...,-9.051327386,...,-9.051327386+-75.5166082,2023 18:00 HORAS+01+23+18:00,0,0,UD UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
92,334,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,22/01/2023,23/01/2023 18:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 432,SECTOR PREVISTO PARO INDEFINIDO SOLICITANDO NU...,-9.092666626,...,-9.092666626+-75.73920822,2023 18:00 HORAS+01+23+18:00,0,0,COE-MTC,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
93,336,SI,VIGENTE,TRÁNSITO INTERRUMPIDO,23/01/2023,23/01/2023 18:00 HORAS,UCAYALI/PADRE ABAD/PADRE ABAD,KM 440,SECTOR NUEVO MUNDO PARO INDEFINIDO SOLICITANDO...,-9.1485618019654,...,-9.1485618019654+-75.759039946864,2023 18:00 HORAS+01+23+18:00,0,0,UNIPRCAR PNP UCAYALI,HUMANO,PE-5N,LONGITUDINAL DE LA SELVA NORTE,SI,interrumpido
